In [1]:
# imports
import sys, os
sys.path.append(os.getcwd()+'/code')
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import Algorithms
from data_streamer import Streamer, MultiStreamer

In [2]:
# constants
train_data_file=os.getcwd() + '/data/train_mnist_normalized.csv'
validation_data_file=os.getcwd() + '/data/test_mnist_normalized.csv'
assert os.path.isfile(train_data_file)
k = 10 # dimensionality of representations
d=784
learning_rate = 1e-3 # initial learning rate
validation_freq = 1000 # how many training steps in between validations

In [3]:
# data streamer and algorithm
train_stream = Streamer(train_data_file).get_stream()
algorithm=Algorithms.rfoja({'d': d, 'k': k, 'learning_rate': learning_rate, 'mean_center': 0.0,'kernel':'rbf','kernel_hyperparameter':0.1,'m':50})
# load validation data at once to make validation faster
validation_data, training_data=[],[]
for point in Streamer(validation_data_file).get_stream():
    validation_data.append(point)
for point in Streamer(train_data_file).get_stream():
    training_data.append(point)
validation_data=np.stack(validation_data,axis=0)
training_data=np.stack(training_data,axis=0)
# do PCA on the validation as a lower bound
centered_data=validation_data #- np.mean(validation_data,axis=0,keepdims=True)
lambdas,_=np.linalg.eigh(np.matmul(centered_data.T,centered_data))
lambdas=sorted(lambdas)
best_validation_loss=sum(lambdas[:-k])/(centered_data.shape[0]*centered_data.shape[1])

In [ ]:
# placeholders for loss
validation_loss, training_loss, min_training_loss = [], [], []
training_batch = []
# main loop
counter=0
train_stream = Streamer(train_data_file).get_stream()
for point in train_stream:
    counter+=1
    algorithm.step(point)
    if counter%validation_freq==0:
        print(counter)
        loss=algorithm.loss(validation_data)
#         loss/=(validation_data.shape[0]*validation_data.shape[1])
        validation_loss.append(loss)
        loss=algorithm.loss(training_data)
#         loss/=(training_data.shape[0]*training_data.shape[1])
        training_loss.append(loss)

1000


In [ ]:
print training_loss

In [ ]:
# plot the train and test loss
plt.figure(figsize=(8, 8))
plt.xlabel('Step')
plt.ylabel('Loss')
x_scale=[x*validation_freq for x in range(1,len(training_loss)+1)]
plt.plot(x_scale, training_loss, '--b', label='train loss')
plt.plot(x_scale, validation_loss, '--r', label='test loss')
plt.plot(x_scale, [best_validation_loss]*len(training_loss), '--k', label='train loss lower bound')
_ = plt.legend()